In [2]:
# Author: Afif Shomali
# Imports
import pandas as pd
import numpy as np
from datetime import datetime

# Pre-Processing Airbnb Data
Source: Inside Airbnb accessed at https://insideairbnb.com/get-the-data/ (Used New York City Datasets, used listings & reviews data)  
License : [Creative Commons Attribution 4.0 International License](#https://creativecommons.org/licenses/by/4.0/)

## Overview of Steps:
- [Merge all data into dataset, removing duplicates by keeping most recent version of a listing](#merging-datasets)
- [Go through and remove uneeded columns](#unecessary-column-removalpruning)
- [Clean up Missing Values in dataset, impute using zero, median or average, remove row completely if needed](#handling-missing-values)
- [Converting datatypes of certain columns, like price & True/False cols, string list to list columns, datetime](#converting-datatypes-of-columns)

After these steps we can move to feature engineering, this work is done in the `FeatureEngineering.ipynb` Notebook.

## Merging Datasets

In [5]:
# # Commented out in order to keep dataset size reasonable
# # Merging Step, don't run this part if only using subset
# df = pd.read_csv("Datasets/AirbnbData/Sep.csv")

# print(df.shape)
# months = ["Aug", "July", "June", "May"]

# for month in months:
#     df_2 = pd.read_csv(f"Datasets/AirbnbData/{month}.csv")
    
#     df = pd.concat([
#         df, 
#         df_2[df_2["id"].isin(df["id"]) == False]
#     ]
#     ).reset_index(drop=True)

# print(df.shape)

# # Checking that we do not have duplicate listings
# len(df["id"].unique())

42432

## Unecessary Column Removal/Pruning 

Columns to Remove:
- listing_url
- scrape_id
- source
- host_url
- host_thumbnail_url
- host_picture_url
- calendar_updated
- calendar_last_scraped
- first_review
- last_review

These don't provide us with any useful information for the goal of creating a predictive model, these columns were selecting after looking over the [data descritpion](https://docs.google.com/spreadsheets/d/1iWCNJcSutYqpULSQHlNyGInUvHg2BoUGoNRIGa6Szc4/edit?gid=1322284596#gid=1322284596), we will likely drop more columns later on after conducting EDA which might uncover more columns as being not impactful.

In [7]:
# Load the combined dataset 
df = pd.read_csv("Datasets/AirbnbData/Full_listings.csv")

# Get an idea of what all the columns are
df.columns

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'source', 'name',
       'description', 'neighborhood_overview', 'picture_url', 'host_id',
       'host_url', 'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'neighbourhood',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm', 'ca

In [8]:
drop_cols = [
    "listing_url",
    "scrape_id",
    "source",
    "host_url",
    "host_thumbnail_url",
    "host_picture_url",
    "calendar_updated",
    "calendar_last_scraped",
    "first_review",
    "last_review",
    "neighbourhood"
]
df.drop(drop_cols, axis=1, inplace=True)

In [9]:
df.columns

Index(['id', 'last_scraped', 'name', 'description', 'neighborhood_overview',
       'picture_url', 'host_id', 'host_name', 'host_since', 'host_location',
       'host_about', 'host_response_time', 'host_response_rate',
       'host_acceptance_rate', 'host_is_superhost', 'host_neighbourhood',
       'host_listings_count', 'host_total_listings_count',
       'host_verifications', 'host_has_profile_pic', 'host_identity_verified',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm', 'has_availability', 'availability_30',
       'availability_60', 'availability_90', 'availability_365',
       'number_of_revi

## Handling Missing Values

We will handle missing values using a variety of techniques, imputing values based on median or mean, filling values with zero or estimating a column based on the values of another column.  
Additionally, If a column has large proportion of N/A values, we may decide drop the column entirely.
The technique use will depend on the column, comments will provide a breif overview as to which technique was used and why.

In [10]:
# Check which columns have N/A values and how many/what percentage
for column in df.columns:
    missing_count = df[column].isna().sum()
    if missing_count > 0:
        print(f'{column}: {missing_count}')

name: 2
description: 1396
neighborhood_overview: 19228
picture_url: 1
host_name: 5
host_since: 5
host_location: 9503
host_about: 18465
host_response_time: 15771
host_response_rate: 15771
host_acceptance_rate: 14948
host_is_superhost: 480
host_neighbourhood: 8924
host_listings_count: 5
host_total_listings_count: 5
host_verifications: 5
host_has_profile_pic: 5
host_identity_verified: 5
bathrooms: 15539
bathrooms_text: 36
bedrooms: 6130
beds: 15703
price: 15544
minimum_minimum_nights: 1
maximum_minimum_nights: 1
minimum_maximum_nights: 1
maximum_maximum_nights: 1
minimum_nights_avg_ntm: 1
maximum_nights_avg_ntm: 1
has_availability: 5487
review_scores_rating: 13422
review_scores_accuracy: 13434
review_scores_cleanliness: 13424
review_scores_checkin: 13438
review_scores_communication: 13429
review_scores_location: 13441
review_scores_value: 13440
license: 36070
reviews_per_month: 13422


In [11]:
# Start with text columns: name, description, neighborhood_overview, host_name, picture_url, host_about
# We will change the missing values to these to just "Blank" (Avoids issues when using None since that makes these values NaN when loading),
#  when we do a sentiment/text quality analysis, we will automatically give these columns the lowest score/default score
df.fillna({
    "name": "Blank",
    "description": "Blank",
    "neighborhood_overview": "Blank",
    "host_name": "Blank",
    "host_about": "Blank"
}, inplace=True)

# Dropping the row without a picture url since there is only 1 row so it won't reduce our dataset size by any significant amount
# This column may or may not be used depending on whether we do feature engineering based on the photo of the airbnb
df.dropna(subset=["picture_url"], inplace=True)

In [12]:
# Next do Boolean/True False Columns: host_is_superhost, host_has_profile_pic, host_identity_verified, has_availability, license

# Columns that will will automatically be false, for these columns, we make the assumption that NaN would be false
# since having no value in these columns would correspond to not having superhost status, or not having a profile picture/license 
df.fillna({
    "host_is_superhost": "f",
    "host_has_profile_pic": "f",
    "license": "No License"
}, inplace=True)

# To fill host_identity_verified, use the host_verifications which lists what methods of verification the host has, 
# if this other column is NaN or an empty list then we make the host_identity_verified column false 
df.loc[df['host_identity_verified'].isna(), 'host_identity_verified'] = df.loc[df['host_identity_verified'].isna(), 'host_verifications'].apply(
    lambda x: "f" if pd.isna(x) or x == '[]' else "t"
)

# To fill has_availability, use availability_30 to check how many days of availabilty there are in the next 30 days, if this number is > 0 then set the value to true
df.loc[df['has_availability'].isna(), 'has_availability'] = df.loc[df['has_availability'].isna(), 'availability_30'].apply(
    lambda x: "t" if x > 0 else 'f'
)

In [13]:
# Then do host related columns that haven't been filled yet: host_since, host_location, host_response_time, host_response_rate, host_acceptance_rate, host_neighbourhood, host_listings_count, host_total_listings_count, host_verifications
# host_verifications, fill NaN with '[]' since NaN means the host hasn't been verified
df.fillna({"host_verifications": '[]'}, inplace=True)

# host_since, set value to date of corresponding last_scraped of the column, since that is when that listing was scraped
df.loc[df['host_since'].isna(), 'host_since'] = df.loc[df['host_since'].isna(), 'last_scraped']

# host_location & host_neighbourhood, if the host neighbourhood is NaN, we fill it with the cleansed listing neighbor hood column
# Then we will fill host_location with New York as this should be the default value for listings without a host location 
# since we are analyzing listings in New York City 
df.loc[df["host_neighbourhood"].isna(), "host_neighbourhood"] = df.loc[df["host_neighbourhood"].isna(), "neighbourhood_cleansed"]

df.fillna({"host_location": "New York, NY"}, inplace=True)

# host_response_time, host_response_rate, host_acceptance_rate, 
# want to set NaN response time to the highest possible which is a "a few days or more"
df.fillna({"host_response_time": "a few days or more"}, inplace=True)

# Want to set  NAN response rate to 0 based on EDA testing this gave us higher correlation between it and predictor variables, 
# Want to do a similar thing for the host acceptance rate
# Make the columns numeric
df["host_response_rate"] = df["host_response_rate"].str.rstrip('%').astype(float)
df["host_acceptance_rate"] = df["host_acceptance_rate"].str.rstrip("%").astype(float)

# Set NaN values to 0 
df.fillna({"host_response_rate" : 0.0,
           "host_acceptance_rate": 0.0}, inplace=True)

# host_listings_count &  host_total_listings_count, set the values to 1 as we can assume that the host only has 1 version of this listing 
# Then use the calculated host listing count to fill host_total listing count
# Might drop these columns after EDA if we see co-linearity with the calculated host listing column
df.fillna({
    "host_listings_count": 1.0
}, inplace=True)

df["host_total_listings_count"] = df["host_total_listings_count"].fillna(df["calculated_host_listings_count"])

In [14]:
# Handle bed & bathroom columns: bathrooms, bathrooms_text, bedrooms, beds
# For bathrooms_text, fill an NaN with "0 baths"
df.fillna({
    "bathrooms_text": "0 baths"
}, inplace=True)

# The for the bathrooms column, if the value is NA, use the bathrooms_text column & extract the first number in the string that appears
# convert to a float and set this as the value of the column, we do this since bathrooms text has less missing values than bathrooms

non_numeric_baths = ['Half-bath', 'Private half-bath', 'Shared half-bath']

df.loc[df["bathrooms"].isna(), "bathrooms"] = df.loc[df["bathrooms"].isna(), "bathrooms_text"].apply(
    lambda x: 0.5 if x in non_numeric_baths else float(x.split()[0])
)

# For bedrooms, use median which is one 
df.fillna({
    "bedrooms": df["bedrooms"].median()
}, inplace=True)

# For beds, if the value is NaN take the number of people the property accomodates and divide it by 2
# Most airbnb listings have a bed being able to accomodate 2 people, so dividing accomodates by 2 will give us a good estimate of the number of beds
df.loc[df["beds"].isna(), "beds"] = df.loc[df["beds"].isna(), "accommodates"].apply(
    lambda x: np.ceil(x / 2)
)

In [ ]:
# Handle rest of numerical columns: price, reviews_score_*, *_nights
# For price, first convert to numerical, then set the price based on the average price of the neighborhood the lisiting is in
df["price"] = df["price"].replace({r'\$': '', r',': ''}, regex=True).astype(float)

# !!! May want to change this to do mean of bourough instead incase certain neigborhoods have a low amount of samples !!!
grouped_means = df.groupby("neighbourhood_cleansed")["price"].transform('mean')
df["price"] = df["price"].fillna(grouped_means)

# Fill last missing price with mean of the bourough
grouped_bourough_means = df.groupby("neighbourhood_group_cleansed")["price"].transform('mean')

df["price"] = df["price"].fillna(grouped_bourough_means)

# for review_scores columns, Fill NA with 0, 
# however we may end up not using these columns since we are going to make an column based on the sentiments of the reivews

df.fillna({
    "review_scores_rating": 0.0,
    "review_scores_accuracy": 0.0,
    "review_scores_cleanliness": 0.0,
    "review_scores_checkin": 0.0,
    "review_scores_communication": 0.0,
    "review_scores_location": 0.0,
    "review_scores_value" : 0.0
}, inplace=True)

# For reviews per month, use number of reviews divided by months since starting to be a host, assume current date is date of last scrape or 9/6/24

current_date = datetime(2024, 9, 6)
df['host_since'] = pd.to_datetime(df['host_since'])


df.loc[df["reviews_per_month"].isna(), "reviews_per_month"] = (
    df.loc[df['reviews_per_month'].isna(), 'number_of_reviews'] / 
    np.ceil(((current_date - df.loc[df['reviews_per_month'].isna(), 'host_since']).dt.days) / 30)
)

# For the max,min, nights columns since there is only one row with missing values we can just remove it
df.dropna(subset=[
    "minimum_minimum_nights",
    "maximum_minimum_nights",
    "minimum_maximum_nights",
    "maximum_maximum_nights",
    "minimum_nights_avg_ntm",
    "maximum_nights_avg_ntm"], inplace=True)

In [17]:
# Check for any remaining NA values
df.isna().sum().sum()

0

## Converting Datatypes of columns

Some of the columns currently have type object, for example price needs to be converted to a numerical object, and the True/False columns need to be converted to Boolean/0,1 columns.

Columns to convert:
- host_is_superhost
- host_has_profile_pic
- host_identity_verified
- has_availability
- instant_bookable
- license (Convert to a 3 catergorical variable, either No license, Exempt, or Has license)

In [18]:
for column in df.columns:
    print(f'{column}: {df[column].dtype}')

id: int64
last_scraped: object
name: object
description: object
neighborhood_overview: object
picture_url: object
host_id: int64
host_name: object
host_since: datetime64[ns]
host_location: object
host_about: object
host_response_time: object
host_response_rate: float64
host_acceptance_rate: float64
host_is_superhost: object
host_neighbourhood: object
host_listings_count: float64
host_total_listings_count: float64
host_verifications: object
host_has_profile_pic: object
host_identity_verified: object
neighbourhood_cleansed: object
neighbourhood_group_cleansed: object
latitude: float64
longitude: float64
property_type: object
room_type: object
accommodates: int64
bathrooms: float64
bathrooms_text: object
bedrooms: float64
beds: float64
amenities: object
price: float64
minimum_nights: int64
maximum_nights: int64
minimum_minimum_nights: float64
maximum_minimum_nights: float64
minimum_maximum_nights: float64
maximum_maximum_nights: float64
minimum_nights_avg_ntm: float64
maximum_nights_avg_n

In [19]:
# Convert Binary Variables t/f to 1/0
bin_cols = [
 "host_is_superhost",
 "host_has_profile_pic",
 "host_identity_verified",
 "has_availability",
 "instant_bookable"
]

df[bin_cols] = df[bin_cols].replace({"t": 1, "f":0})

C:\Users\Afif\AppData\Local\Temp\ipykernel_41972\3547918659.py:10: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[bin_cols] = df[bin_cols].replace({"t": 1, "f":0})


In [20]:
# Convert license 
df["license"] = df["license"].apply(lambda x: x if x == "No License" or x == "Exempt" else "Has License")

In [21]:
# Save Final Dataset
# df.to_csv("Datasets/AirbnbData/All_Listings_Cleaned.csv", index=False)